Importations of libs

In [0]:
import cv2
import numpy as np
from google.colab import files
import random
import math
import os
import pickle
import tensorflow.keras.models as km
import tensorflow.keras.layers as layers
import tensorflow as tf
import tensorflow.keras.callbacks as call
import tensorflow.keras.applications as ka
from IPython.display import clear_output
import tensorflow.keras.regularizers as kr


Functions to connect to drive to take the dataset

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Verification that it's well connected

In [0]:
os.getcwd()

'/content'

To see what pickle files exist

In [0]:
path_pickle = os.getcwd() + "/gdrive/My Drive/dogs_AI/pickles"
os.listdir(path_pickle)

['images_6_1.pickle',
 'labels_6_1.pickle',
 'images_6_2.pickle',
 'labels_6_2.pickle',
 'images_6_3.pickle',
 'labels_6_3.pickle',
 'images_6_4.pickle',
 'labels_6_4.pickle']

Fonctions for categories: one for all, one for only 6 categories

In [0]:
# for all breeds
def get_categories():
    """ To have all categories names.

     Return:
         categories: a list of all categories
     """
    # Take the path to directory "dataset"
    path = os.getcwd()
    path = path + "/gdrive/My Drive/dataset"
    dataset_dirs = os.listdir(path)
    # Declare the list which will contain categories names
    categories = list()

    # Browse the "dataset" directory to get all directories names which are the categories
    for dirs in dataset_dirs:
        categories.append(dirs)

    return categories


In [0]:
# for 6 breeds
def get_categories_6_breeds():
    """ To have all categories names.

     Return:
         categories: a list of all categories
     """
    # Take the path to directory "dataset"
    path = os.getcwd()
    path = path + "/gdrive/My Drive/dataset"
    dataset_dirs = os.listdir(path)
    # Declare the list which will contain categories names
    categories = list()

    # Browse the "dataset" directory to get all directories names which are the categories
    for dirs in dataset_dirs:
      if (dirs == "pug" or dirs == "English_setter" or dirs == "German_shepherd"
                    or dirs == "golden_retriever" or dirs == "collie" or dirs == "boxer"):
        categories.append(dirs)

    return categories

Load all constants

In [0]:
# load constants
# CATEGORIES = get_categories()  # we take all the categories at first
CATEGORIES = get_categories_6_breeds()  # we take all the categories at first
SIZE = 250  # size of images: SIZE * SIZE
LEARNING_RATE = 5e-8  # the model's learning rate
BATCH_SIZE = 1  # the batch size of data to train the model, 36 is good
MODEL_NAME = "dogs_6"  # the name that will be taken for the model's save
GRAY_SCALE = False  # to know if we use grayscale images  to feed the model
DOGS_6 = True  # to know if we take only 6 breeds of dogs to test the model or all breeds
STEPS_PER_EPOCH = 129  # steps per epoch, 100 is good for large dataset, else use less
EPOCHS = 6  # number of epochs to train the model

Verification that the categories have been loaded

In [0]:
CATEGORIES

['pug',
 'English_setter',
 'golden_retriever',
 'German_shepherd',
 'collie',
 'boxer']

Functions to save all dataset in pickle file to avoid preprocessing again

In [0]:
def save_files_cv2(categories, size, gray_scale):
    """ Get files only for 6 categories (golden retriever, english setter,
    german shepard, collie, pug) to test if the model works with a few breeds """
    # Take the path to directory "dataset"
    path = os.getcwd()
    path = path + "/gdrive/My Drive/dataset"
    path_pickle = os.getcwd() + "/gdrive/My Drive/dogs_AI/pickles"
    dataset_dirs = os.listdir(path)
    # Declare the two list which will contain labels and path of the images
    labels = list()
    images = list()
    
    # counter for save batch every 1 000 images
    i = 0
    j = 1

    # Browse the "dataset" directory to get all paths for images and labels which are the directories names
    for dirs in dataset_dirs:
        dir_path = path + "/" + dirs
        for path2, dir, files in os.walk(dir_path):
            for files_name in files:
                labels.append(categories.index(dirs))
                img = cv2_preprocess(dir_path + "/" + files_name, size, gray_scale)
                images.append(img.tolist())

                # Data augmentation
                # flip the image
                img_flipped = np.fliplr(img)
                labels.append(categories.index(dirs))
                images.append(img_flipped.tolist())

                if random.random() < 0.6:
                    img = data_left_translation(img, size)
                    labels.append(categories.index(dirs))
                    images.append(img.tolist())
                elif random.random() < 0.6:
                    img = data_right_translation(img, size)
                    labels.append(categories.index(dirs))
                    images.append(img.tolist())
                    
                img = None
                    
                if i >= 500:
                    pickle_out = open("{}/images_6_{}.pickle".format(path_pickle, j), "wb")
                    pickle.dump(images, pickle_out)
                    pickle_out.close()
                    
                    images = None

                    pickle_out = open("{}/labels_6_{}.pickle".format(path_pickle, j), "wb")
                    pickle.dump(labels, pickle_out)
                    pickle_out.close()
                    
                    labels = None
                    
                    images = list()
                    labels = list()
                        
                    i = 0
                    print("end of j == {} in all breeds".format(j))
                    j += 1

    if i != 0:
      pickle_out = open("{}/images_6_{}.pickle".format(path_pickle, j), "wb")
      pickle.dump(images, pickle_out)
      pickle_out.close()
      
      images = None

      pickle_out = open("{}/labels_6_{}.pickle".format(path_pickle, j), "wb")
      pickle.dump(labels, pickle_out)
      pickle_out.close()
      
      labels = None
                        
      i = None
      j = None
      
      print(" end of all breeds ")


def save_files_for_some_categories_cv2(categories, size, gray_scale):
    """ Get files only for 6 categories (golden retriever, english setter,
    german shepard, collie, pug) to test if the model works with a few breeds """
    # Take the path to directory "dataset"
    path = os.getcwd()
    path = path + "/gdrive/My Drive/dataset"
    
    path_pickle = os.getcwd() + "/gdrive/My Drive/dogs_AI/pickles"
    
    dataset_dirs = os.listdir(path)
    # Declare the two list which will contain labels and path of the images
    labels = list()
    images = list()
    
    # counter for save batch every 1 000 images
    i = 0
    j = 1

    # Browse the "dataset" directory to get all paths for images and labels which are the directories names
    for dirs in dataset_dirs:
        dir_path = path + "/" + dirs
        for path2, dir, files in os.walk(dir_path):
            if (dirs == "pug" or dirs == "English_setter" or dirs == "German_shepherd"
                    or dirs == "golden_retriever" or dirs == "collie" or dirs == "boxer"):
                for files_name in files:
                    labels.append(categories.index(dirs))
                    img = cv2_preprocess(dir_path + "/" + files_name, size, gray_scale)
                    images.append(img.tolist())
                    
                    i += 1

                    # Data augmentation
                    if random.random() < 0.4:
                        # flip the image
                        img = np.fliplr(img)
                        labels.append(categories.index(dirs))
                        images.append(img.tolist())
                        i += 1

                    if random.random() < 0.2:
                        img = data_left_translation(img, size)
                        labels.append(categories.index(dirs))
                        images.append(img.tolist())
                        i += 1
                    elif random.random() < 0.2:
                        img = data_right_translation(img, size)
                        labels.append(categories.index(dirs))
                        images.append(img.tolist())
                        i += 1
                    
                    img = None
                    
                    if i >= 400:
                        pickle_out = open("{}/images_6_{}.pickle".format(path_pickle, j), "wb")
                        pickle.dump(images, pickle_out)
                        pickle_out.close()

                        pickle_out = open("{}/labels_6_{}.pickle".format(path_pickle, j), "wb")
                        pickle.dump(labels, pickle_out)
                        pickle_out.close()
                        
                        images = None
                        images = list()
                        labels = None
                        labels = list()
                        
                        i = 0
                        print("end of j == {} in 6 breeds".format(j))
                        j += 1
    
    if i != 0:
      pickle_out = open("{}/images_6_{}.pickle".format(path_pickle, j), "wb")
      pickle.dump(images, pickle_out)
      pickle_out.close()

      pickle_out = open("{}/labels_6_{}.pickle".format(path_pickle, j), "wb")
      pickle.dump(labels, pickle_out)
      pickle_out.close()
                        
      images = None
      labels = None
                        
      i = None
      j = None
      
      print(" end of 6 breeds")
    
    

In [0]:
def cv2_preprocess(path, size, gray_scale):
    # channels = 1 if gray_scale else 3

    if gray_scale:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    else:
        img = cv2.imread(path)

    img = cv2.resize(img, (size, size))
    # img = np.array(img).reshape(-1, 192, 192, channels)
    img = img / 255.0
    return img


def data_left_translation(img, size):
    for i in range(size, 1, -1):
        for j in range(size):
            if i < size - 20:
                img[j][i] = img[j][i-20]
            elif i < size-1:
                img[j][i] = 0
    return img


def data_right_translation(img, size):
    for i in range(size, 1, -1):
        for j in range(size):
            if i < size - 20:
                img[j][i] = img[j][i+20]
    return img

Launch the save for only 6 categories (be aware to choose good constants ofr only 6 categories)

In [0]:
save_files_for_some_categories_cv2(get_categories_6_breeds(), SIZE, False)

end of j == 1 in 6 breeds
end of j == 2 in 6 breeds
end of j == 3 in 6 breeds
end of j == 4 in 6 breeds
 end of 6 breeds


Fonction to save the model after the training

In [0]:
def save_model(model, name):
    """ Save a given model with a particular name.

         Args:
             model: the model to save
             name: the name to use to save the model
         """
    path = os.getcwd()
    path = path + "/gdrive/My Drive/dogs_AI/models/" + name + ".model"
    model.save(path)

To load every pickle files with pictures into a numpy array

In [0]:
# better loading to test

images_list = list()
labels_list = list()
labels = list()

path_pickle = os.getcwd() + "/gdrive/My Drive/dogs_AI/pickles/"

for i in range(5):
  pickle_in = open("{}images_6_{}.pickle".format(path_pickle, i+1), "rb")
  images_list = pickle.load(pickle_in)
  pickle_in.close()

  pickle_in = open("{}labels_6_{}.pickle".format(path_pickle, i+1), "rb")
  labels_list = pickle.load(pickle_in)
  pickle_in.close()
  
  images_list = np.array(images_list)
  labels_list = np.array(labels_list)
  
  if i == 0:
    images = np.array(images_list)
    labels = np.array(labels_list)
  else:
    images = np.concatenate((images, images_list))
    labels = np.concatenate((labels, labels_list))
    
  images_list = None
  labels_list = None
  
  
  print(i)
  

randomize = np.arange(len(labels))
np.random.shuffle(randomize)
images = images[randomize]
labels = labels[randomize]

images = images.reshape(-1, SIZE, SIZE, 3)

0
1
2
3
4


To see the shape of the numpy array containing all the pictures

In [0]:
print(images.shape)
print(len(labels))

(1698, 250, 250, 3)
1698


In [0]:
images = list()
labels = list()

path_pickle = os.getcwd() + "/gdrive/My Drive/dogs_AI/pickles/"

for i in range(2):
  pickle_in = open("{}images_6_{}.pickle".format(path_pickle, i+1), "rb")
  images = images + pickle.load(pickle_in)
  pickle_in.close()

  pickle_in = open("{}labels_6_{}.pickle".format(path_pickle, i+1), "rb")
  labels = labels + pickle.load(pickle_in)
  pickle_in.close()
  
  print(i)
  
combine = list(zip(images, labels))
random.shuffle(combine)
images, labels = zip(*combine)
combine = None
images = np.array(images).reshape(-1, SIZE, SIZE, 3)


0
1


In [0]:
images.shape

(1906, 200, 200, 3)

If there is an error, to empty all variables

In [0]:
images = None
labels = None
model = None

In [0]:
model = None

In [0]:
model = train(batch_size = 20, epochs=40, model_name="tryNewPhotos")

Train on 1524 samples, validate on 382 samples
Epoch 1/40
1524/1524 [==============================] - 7s 4ms/sample - loss: 1.8277 - acc: 0.2001 - val_loss: 1.8013 - val_acc: 0.1806
Epoch 2/40
1524/1524 [==============================] - 6s 4ms/sample - loss: 1.7923 - acc: 0.2211 - val_loss: 1.8072 - val_acc: 0.1806
Epoch 3/40
1524/1524 [==============================] - 6s 4ms/sample - loss: 1.7863 - acc: 0.2264 - val_loss: 1.7926 - val_acc: 0.1806
Epoch 4/40
1524/1524 [==============================] - 6s 4ms/sample - loss: 1.7872 - acc: 0.2251 - val_loss: 1.7852 - val_acc: 0.1859
Epoch 5/40
1524/1524 [==============================] - 6s 4ms/sample - loss: 1.7699 - acc: 0.2231 - val_loss: 1.8006 - val_acc: 0.1806
Epoch 6/40
1524/1524 [==============================] - 6s 4ms/sample - loss: 1.7789 - acc: 0.2395 - val_loss: 1.7918 - val_acc: 0.1806
Epoch 7/40
1524/1524 [==============================] - 6s 4ms/sample - loss: 1.7457 - acc: 0.2408 - val_loss: 1.7637 - val_acc: 0.2042
E

Normal training function with test with convutional layers

In [0]:
 # training function
def train(gray_scale=False, epochs=10, batch_size=32, model_name="dogs_colab", dogs_6=False):
    """ Train the model with data contained in the dataset directory.

    Args:
        learning_rate: the learning rate used by the optimizer of the model
        gray_scale: boolean to know if we need to use grayscale image for inputs
        epochs: number of epochs to train the model, 5 by default
        batch_size: the batch size for data to train the model, 20 by default
        steps_per_epoch: number of steps per epoch, 100 by default
        model_name: the name used to save the model, dogs by default
        dogs_6: boolean to know if we need to use only 6 breeds to test the model
    Return:
        a trained model
    """

    model = km.Sequential()
    model.add(layers.Conv2D(32, (4, 4), input_shape=(SIZE, SIZE, 3), activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(64, (4, 4), activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(128, (4, 4), activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(256, (4, 4), activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation="relu"))
    model.add(layers.Dense(50, activation="relu"))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(len(CATEGORIES), activation="softmax", kernel_regularizer=kr.l2(0.001)))

    model.compile(
        optimizer="adam",
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.fit(images, labels, batch_size=batch_size, epochs=epochs, validation_split=0.2)

    save_model(model, model_name)

    return model

Same but with a tensorboard option

In [0]:
# training function for optimization with tensorboard
def train_tensorboard(conv, dense, neurons, regularization, epochs=10, batch_size=32, model_name="dogs", dogs_6=False):
    """ Train the model with data contained in the dataset directory.

    Args:
        learning_rate: the learning rate used by the optimizer of the model
        gray_scale: boolean to know if we need to use grayscale image for inputs
        epochs: number of epochs to train the model, 5 by default
        batch_size: the batch size for data to train the model, 20 by default
        steps_per_epoch: number of steps per epoch, 100 by default
        model_name: the name used to save the model, dogs by default
        dogs_6: boolean to know if we need to use only 6 breeds to test the model
    Return:
        a trained model
    """
    
    clear_output()
    print(model_name)
    
    model = km.Sequential()
    model.add(layers.Conv2D(32, (4, 4), input_shape=(SIZE, SIZE, 3)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))
    model.add(layers.Activation("relu"))
    
    for l in range(conv-1):
      model.add(layers.Conv2D(32*(l+1), (4, 4)))
      model.add(layers.Activation("relu"))
      model.add(layers.MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))
      model.add(layers.Dropout(0.2))
  
                
    for l in range(dense):
      model.add(layers.Flatten())
      model.add(layers.Dense(neurons*(dense+1-l), activation="relu"))
      model.add(layers.Dropout(0.3))
    

    model.add(layers.Flatten())
    model.add(layers.Dense(len(CATEGORIES), activation="softmax", kernel_regularizer=kr.l2(regularization)))

    model.compile(
        optimizer="adam",
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(images, labels, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=[tensorboard])

    save_model(model, model_name)

    return model

Loop to test different options

In [0]:
path_log = os.getcwd() + "/gdrive/My Drive/dogs_AI/logs/try2/"



# listes pour changer l'entrainement
regularization = [0.0001, 0.001]
conv2D_nb = [2, 3, 4, 5]
dense_nb = [1, 2, 3]
neurons_dense = [50, 100, 200]
# boucle for pour entrainer
for neurons in neurons_dense:
  for dense in dense_nb:
    for conv in conv2D_nb:
      for reg in regularization:
        name_log = "log_conv={}_dense={}_neurons={}_regu={}".format(conv, dense, neurons, reg)

        # lancement tensorboard
        tensorboard = call.TensorBoard(log_dir=path_log + '{}'.format(name_log))

        train_tensorboard(conv, dense, neurons, reg, model_name=name_log, batch_size=18, epochs=15)
      

log_conv=2_dense=3_neurons=50_regu=0.0001
Train on 1524 samples, validate on 382 samples


KeyboardInterrupt: ignored

In [0]:
files.download('images.pickle')
files.download('labels.pickle')
files.download('images_6.pickle')
files.download('labels_6.pickle')

Function to load a model

In [0]:
def load_model(name):
  path = os.getcwd() + "/gdrive/My Drive/dogs_AI/models/" + name + ".model"
  return km.load_model(path)

To test a model on a test photo

In [0]:
model = load_model("tryNewPhotos")

path = os.getcwd() + "/gdrive/My Drive/dogs_AI/photo_test/blago.jpg"

image = cv2_preprocess(path, SIZE, False)
# print(image)
image = np.array(image).reshape(1, SIZE, SIZE, 3)

prediction = model.predict([image])

prediction1 = np.argmax(prediction)
prediction1_pourc = round(prediction[0][prediction1]*100, 2)
prediction1 = CATEGORIES[prediction1]
print("{} à {}%".format(prediction1, prediction1_pourc))

collie à 93.71%


In [0]:
prediction*100

array([[ 0.5988873 ,  4.4927077 ,  0.33432558,  0.2872912 , 93.712845  ,
         0.57394195]], dtype=float32)

In [0]:
CATEGORIES

['pug',
 'English_setter',
 'golden_retriever',
 'German_shepherd',
 'collie',
 'boxer']

In [0]:
# transfer learning with Inception

def train_inception(dense, neurons, regularization, epochs=10, batch_size=32, model_name="dogs_inceptionV3", dogs_6=False):
    """ Train the model with data contained in the dataset directory.

    Args:
        learning_rate: the learning rate used by the optimizer of the model
        gray_scale: boolean to know if we need to use grayscale image for inputs
        epochs: number of epochs to train the model, 5 by default
        batch_size: the batch size for data to train the model, 20 by default
        steps_per_epoch: number of steps per epoch, 100 by default
        model_name: the name used to save the model, dogs by default
        dogs_6: boolean to know if we need to use only 6 breeds to test the model
    Return:
        a trained model
    """
    
    clear_output()
    print(model_name)
    
    conv_base = ka.InceptionV3(include_top=False, weights="imagenet", input_shape=(SIZE, SIZE, 3))
    
    model = km.Sequential()
    model.add(conv_base)
    model.add(layers.GlobalAveragePooling2D())
    
    # model.add(layers.Flatten())
  
                
    # for l in range(dense):
    #   model.add(layers.Dense(neurons*(dense+1-l), activation="relu"))
    #   model.add(layers.Dropout(0.3))
      
    
    model.add(layers.Dense(len(CATEGORIES), activation="softmax", kernel_regularizer=kr.l2(regularization)))
    
    model.summary()

    model.compile(
        optimizer="adam",
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(images, labels, batch_size=batch_size, epochs=epochs, validation_split=0.2)

    save_model(model, model_name)

    return model

In [0]:
# transfer learning with ResNet

def train_resnet(dense, neurons, regularization, epochs=10, batch_size=32, model_name="dogs_inceptionV3", dogs_6=False):
    """ Train the model with data contained in the dataset directory.

    Args:
        learning_rate: the learning rate used by the optimizer of the model
        gray_scale: boolean to know if we need to use grayscale image for inputs
        epochs: number of epochs to train the model, 5 by default
        batch_size: the batch size for data to train the model, 20 by default
        steps_per_epoch: number of steps per epoch, 100 by default
        model_name: the name used to save the model, dogs by default
        dogs_6: boolean to know if we need to use only 6 breeds to test the model
    Return:
        a trained model
    """
    
    clear_output()
    print(model_name)
    
    conv_base = ka.InceptionResNetV2(include_top=False, weights="imagenet", input_shape=(SIZE, SIZE, 3))
    
    model = km.Sequential()
    model.add(conv_base)
    model.add(layers.GlobalAveragePooling2D())
    
    # model.add(layers.Flatten())
  
                
    # for l in range(dense):
    #   model.add(layers.Dense(neurons*(dense+1-l), activation="relu"))
    #   model.add(layers.Dropout(0.3))
      
    
    model.add(layers.Dense(len(CATEGORIES), activation="softmax", kernel_regularizer=kr.l2(regularization)))
    
    model.summary()

    model.compile(
        optimizer="adam",
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(images, labels, batch_size=batch_size, epochs=epochs, validation_split=0.2)

    save_model(model, model_name)

    return model

In [0]:
# transfer learning with Inception

# training function for optimization with tensorboard
def train_inception_tensorboard(dense, neurons, regularization, epochs=10, batch_size=32, model_name="dogs_inceptionV3", dogs_6=False):
    """ Train the model with data contained in the dataset directory.

    Args:
        learning_rate: the learning rate used by the optimizer of the model
        gray_scale: boolean to know if we need to use grayscale image for inputs
        epochs: number of epochs to train the model, 5 by default
        batch_size: the batch size for data to train the model, 20 by default
        steps_per_epoch: number of steps per epoch, 100 by default
        model_name: the name used to save the model, dogs by default
        dogs_6: boolean to know if we need to use only 6 breeds to test the model
    Return:
        a trained model
    """
    
    clear_output()
    print(model_name)
    
    conv_base = ka.InceptionV3(include_top=False, weights="imagenet", input_shape=(SIZE, SIZE, 3))
    
    model = km.Sequential()
    model.add(conv_base)
    model.add(layers.GlobalAveragePooling2D())
    
    model.add(layers.Flatten())
  
                
    for l in range(dense):
      model.add(layers.Dense(neurons*(dense+1-l), activation="relu"))
      model.add(layers.Dropout(0.3))
      
    
    model.add(layers.Dense(len(CATEGORIES), activation="softmax", kernel_regularizer=kr.l2(regularization)))
    
    model.summary()

    model.compile(
        optimizer="adam",
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(images, labels, batch_size=batch_size, epochs=epochs, validation_split=0.3, callbacks=[tensorboard])

    save_model(model, model_name)

    return model

In [0]:
dense = 1
neurons = 100
reg = 0.00004
name_log = "log_dense={}_neurons={}_regu={}".format(dense, neurons, reg)

train_inception(dense, neurons, reg, model_name=name_log, batch_size=10, epochs=20)

log_dense=1_neurons=100_regu=4e-05
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
87916544/87910968 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 6, 6, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 12294     
Total params: 21,815,078
Trainable params: 21,780,646
Non-trainable params: 34,432
_________________________________________________________________
Train on 1358 samples, validate on 340 samples
Epoch 1/20
1358/1358 [==============================] - 61s 45ms/sample - loss: 1.2922 - acc: 0.5221 - v

In [0]:
dense = 1
neurons = 100
reg = 0.000005
name_log = "log_dense={}_neurons={}_regu={}".format(dense, neurons, reg)

train_resnet(dense, neurons, reg, model_name=name_log, batch_size=10, epochs=20)

log_dense=1_neurons=100_regu=1e-05


In [0]:
# Try multiple args for Inception Model

path_log = os.getcwd() + "/gdrive/My Drive/dogs_AI/logs/tryIncep2/"



# listes pour changer l'entrainement
regularization = [0.0001, 0.0003, 0.0005, 0.001]
dense_nb = [1]
neurons_dense = [100, 120]
# boucle for pour entrainer
for neurons in neurons_dense:
  for dense in dense_nb:
      for reg in regularization:
        name_log = "log_dense={}_neurons={}_regu={}".format(dense, neurons, reg)

        # lancement tensorboard
        tensorboard = call.TensorBoard(log_dir=path_log + '{}'.format(name_log))

        train_inception_tensorboard(dense, neurons, reg, model_name=name_log, batch_size=20, epochs=25)
        
        model = None

log_dense=2_neurons=100_regu=0.0005
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 4, 4, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_11  (None, 2048)              0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dense_27 (Dense)             (None, 300)               614700    
_________________________________________________________________
dropout_16 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 200)               60200     
_________________________________________________________________
dropout_17 (Dropo